What’s wrong?

Always returns zenith…

1. Could be due to indexing, but I did an equation for this, along with working on it today

2. Could be because I no longer have those conditions

3. Still at zero after starting loc at a different val, so that’s not the issue.


4. Some inherent issue in mesh grids :) 
It’s definitely something simple, but I haven’t found it


I could resolve 2 by figuring out how to replace values in an array, need to be able to go into specific indices, and I’m not sure if there’s a way to do that without iterating, rendering this entire thing worthless. 


1. Isn’t it either apparently, I tried manually checking this so again not confident this is the problem. 


In [2]:

#Needs to know how to point to healpy 
import healpy as hp

class Sky():
    """
    Generates an array of GRB's given 
    certains strength at different sky positions.
    
    Output should be an array. 
    """
    def __init__(self,NSIDE,strength):

        #depending on NSIDE, there will be anywhere from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = hp.nside2npix(NSIDE)

        #want to convert these pixels into theta phi coords. 
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(hp.pix2ang(NSIDE,i))

    def say_Ao(self):
        """
        """

        print("The GRBs being tested will be " + str(self.Ao) + " counts strong.")
        


In [3]:
import numpy as np
import healpy as hp
import burstutils as bf
import random as rand
import statistics as s

class BurstCube():

    def __init__(self,background,dettilt,alternating=False):
        if alternating == False:
            self.tilt = np.deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
        
            self.tiltC = self.tiltA = dettilt
            self.tiltD = self.tiltB
        self.zenith = [0 , 0]
        self.bg = background

            
    
    #make the normal vectors!
        #self.detA = [ zenith[0] + self.tiltA , zenith[1] ] 
        #self.detB = [ zenith[0] + self.tiltB , zenith[1] + np.pi/2 ] 
       # self.detC = [ zenith[0] + self.tiltC , zenith[1] + np.pi ] 
        #self.detD = [ zenith[0] + self.tiltD , zenith[1] + 3*np.pi/2 ] 

        #self.Anorm = hp.ang2vec(self.detA[0],self.detA[1])
        #self.Bnorm = hp.ang2vec(self.detB[0],self.detB[1])
        #self.Cnorm = hp.ang2vec(self.detC[0],self.detC[1])
        #self.Dnorm = hp.ang2vec(self.detD[0],self.detD[1])

    
       # self.dets = [self.Anorm,self.Bnorm,self.Cnorm,self.Dnorm] 
        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB):   #is this how I inherit? 
        #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        
        self.localizationerrors = []    
        for i in range(len(GRB.sourceangs)):
            sourceAng = GRB.sourceangs[i]
            print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<1:
                sepA=bf.angle(sourcexyz,self.normA)
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                if sepA < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryA=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normA))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryA = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsA = dtheoryA + self.bg #another artifact, incl this background effect somewhere
                unccountsA = np.sqrt(countsA)
                detactualA = rand.gauss(countsA,unccountsA)  #there is a lot of noise, present, updating it now. 
                if detactualA-self.bg < 0:
                    detactualA = self.bg
                    
                detcountsA = detactualA
                
                sepB=bf.angle(sourcexyz,self.normB)
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                if sepB < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryB=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normB))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryB = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsB = dtheoryB + self.bg #another artifact, incl this background effect somewhere
                unccountsB = np.sqrt(countsB)
                detactualB = rand.gauss(countsB,unccountsB)  #there is a lot of noise, present, updating it now. 
                if detactualB-self.bg < 0:
                    detactualB = self.bg
                    
                detcountsB = detactualB

                sepC=bf.angle(sourcexyz,self.normC)
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                if sepC < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryC=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normC))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryC = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsC = dtheoryC + self.bg #another artifact, incl this background effect somewhere
                unccountsC = np.sqrt(countsC)
                detactualC = rand.gauss(countsC,unccountsC)  #there is a lot of noise, present, updating it now. 
                if detactualC-self.bg < 0:
                    detactualC = self.bg
                    
                detcountsC = detactualC
                
                sepD=bf.angle(sourcexyz,self.normD)
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                if sepD < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryD=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normD))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryD = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsD = dtheoryD + self.bg #another artifact, incl this background effect somewhere
                unccountsD = np.sqrt(countsD)
                detactualD = rand.gauss(countsD,unccountsD)  #there is a lot of noise, present, updating it now. 
                if detactualD-self.bg < 0:
                    detactualD = self.bg
                    
                detcountsD = detactualD
                
                
                chiA = bf.quad_solver(detcountsA,self.normA,0,90,0,360,self.bg)
                chiB = bf.quad_solver(detcountsB,self.normB,0,90,0,360,self.bg)
                chiC = bf.quad_solver(detcountsC,self.normC,0,90,0,360,self.bg)
                chiD = bf.quad_solver(detcountsD,self.normD,0,90,0,360,self.bg)
                
                chisquared = np.add(np.add(chiA,chiB),np.add(chiC,chiD)) #adds it all up for total chi2
                
                
                #now obtain thetaloc
                chimin=np.amin(chisquared)
                chisquareds=list(chisquared)
                print("Index of chimin " + str(chisquareds.index(chimin)))
               # oa = np.linspace(0,90,30)
               # ob = np.linspace(0,360,40)
                #Aofit = np.linspace(0,10,50)
                #thetaloc = np.rad2deg(oa[int((chisquareds.index(chimin)-(chisquareds.index(chimin) % (len(ob)*len(Aofit))))/(len(ob)*len(Aofit)))])
                #print("theta loc: " + str(thetaloc))
                loop +=1
        return thetaloc

In [4]:
GRB1 = Sky(8,500)

In [5]:
cube1= BurstCube(1000,45)

In [6]:
cube1.response2GRB(GRB1)

Testing [  5.85026706  45.        ]
Index of chimin 0
Testing [   5.85026706  135.        ]
Index of chimin 1
Testing [   5.85026706  225.        ]
Index of chimin 1
Testing [   5.85026706  315.        ]
Index of chimin 1
Testing [ 11.71585239  22.5       ]
Index of chimin 21
Testing [ 11.71585239  67.5       ]
Index of chimin 0
Testing [  11.71585239  112.5       ]
Index of chimin 0
Testing [  11.71585239  157.5       ]
Index of chimin 0
Testing [  11.71585239  202.5       ]
Index of chimin 21
Testing [  11.71585239  247.5       ]
Index of chimin 0
Testing [  11.71585239  292.5       ]
Index of chimin 1
Testing [  11.71585239  337.5       ]
Index of chimin 0
Testing [ 17.61243907  15.        ]
Index of chimin 21
Testing [ 17.61243907  45.        ]
Index of chimin 0
Testing [ 17.61243907  75.        ]
Index of chimin 0
Testing [  17.61243907  105.        ]
Index of chimin 21
Testing [  17.61243907  135.        ]
Index of chimin 0
Testing [  17.61243907  165.        ]
Index of chimin 1


Index of chimin 8
Testing [ 60.      5.625]
Index of chimin 8
Testing [ 60.     16.875]
Index of chimin 8
Testing [ 60.     28.125]
Index of chimin 8
Testing [ 60.     39.375]
Index of chimin 11
Testing [ 60.     50.625]
Index of chimin 0
Testing [ 60.     61.875]
Index of chimin 8
Testing [ 60.     73.125]
Index of chimin 10
Testing [ 60.     84.375]
Index of chimin 10
Testing [ 60.     95.625]
Index of chimin 10
Testing [  60.     106.875]
Index of chimin 10
Testing [  60.     118.125]
Index of chimin 10
Testing [  60.     129.375]
Index of chimin 10
Testing [  60.     140.625]
Index of chimin 10
Testing [  60.     151.875]
Index of chimin 10
Testing [  60.     163.125]
Index of chimin 10
Testing [  60.     174.375]
Index of chimin 0
Testing [  60.     185.625]
Index of chimin 12
Testing [  60.     196.875]
Index of chimin 12
Testing [  60.     208.125]
Index of chimin 12
Testing [  60.     219.375]
Index of chimin 12
Testing [  60.     230.625]
Index of chimin 19
Testing [  60.     

Index of chimin 12
Testing [  90.     230.625]
Index of chimin 19
Testing [  90.     241.875]
Index of chimin 12
Testing [  90.     253.125]
Index of chimin 18
Testing [  90.     264.375]
Index of chimin 12
Testing [  90.     275.625]
Index of chimin 18
Testing [  90.     286.875]
Index of chimin 12
Testing [  90.     298.125]
Index of chimin 18
Testing [  90.     309.375]
Index of chimin 8
Testing [  90.     320.625]
Index of chimin 8
Testing [  90.     331.875]
Index of chimin 8
Testing [  90.     343.125]
Index of chimin 8
Testing [  90.     354.375]
Index of chimin 8
Testing [ 94.78019185   0.        ]
Index of chimin 17
Testing [ 94.78019185  11.25      ]
Index of chimin 8
Testing [ 94.78019185  22.5       ]
Index of chimin 8
Testing [ 94.78019185  33.75      ]
Index of chimin 8
Testing [ 94.78019185  45.        ]
Index of chimin 8
Testing [ 94.78019185  56.25      ]
Index of chimin 18
Testing [ 94.78019185  67.5       ]
Index of chimin 10
Testing [ 94.78019185  78.75      ]
Index

Index of chimin 8
Testing [ 120.      95.625]
Index of chimin 8
Testing [ 120.     106.875]
Index of chimin 18
Testing [ 120.     118.125]
Index of chimin 8
Testing [ 120.     129.375]
Index of chimin 18
Testing [ 120.     140.625]
Index of chimin 16
Testing [ 120.     151.875]
Index of chimin 18
Testing [ 120.     163.125]
Index of chimin 12
Testing [ 120.     174.375]
Index of chimin 18
Testing [ 120.     185.625]
Index of chimin 16
Testing [ 120.     196.875]
Index of chimin 16
Testing [ 120.     208.125]
Index of chimin 16
Testing [ 120.     219.375]
Index of chimin 8
Testing [ 120.     230.625]
Index of chimin 18
Testing [ 120.     241.875]
Index of chimin 18
Testing [ 120.     253.125]
Index of chimin 8
Testing [ 120.     264.375]
Index of chimin 18
Testing [ 120.     275.625]
Index of chimin 12
Testing [ 120.     286.875]
Index of chimin 12
Testing [ 120.     298.125]
Index of chimin 11
Testing [ 120.     309.375]
Index of chimin 18
Testing [ 120.     320.625]
Index of chimin 16

Index of chimin 12
Testing [ 150.43443884  243.        ]
Index of chimin 10
Testing [ 150.43443884  261.        ]
Index of chimin 18
Testing [ 150.43443884  279.        ]
Index of chimin 10
Testing [ 150.43443884  297.        ]
Index of chimin 16
Testing [ 150.43443884  315.        ]
Index of chimin 9
Testing [ 150.43443884  333.        ]
Index of chimin 16
Testing [ 150.43443884  351.        ]
Index of chimin 12
Testing [ 156.44353569   11.25      ]
Index of chimin 18
Testing [ 156.44353569   33.75      ]
Index of chimin 12
Testing [ 156.44353569   56.25      ]
Index of chimin 8
Testing [ 156.44353569   78.75      ]
Index of chimin 16
Testing [ 156.44353569  101.25      ]
Index of chimin 12
Testing [ 156.44353569  123.75      ]
Index of chimin 8
Testing [ 156.44353569  146.25      ]
Index of chimin 11
Testing [ 156.44353569  168.75      ]
Index of chimin 12
Testing [ 156.44353569  191.25      ]
Index of chimin 12
Testing [ 156.44353569  213.75      ]
Index of chimin 8
Testing [ 156.44

NameError: name 'thetaloc' is not defined

In [ ]:
a

### a

In [ ]:
b = np.array([1,2,3,4,5])

In [ ]:
mask = b<3

In [ ]:
b[mask]

In [ ]:
b = b[mask]

In [ ]:
b